In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from itertools import product
import sys


In [ ]:
sns.set(style="whitegrid")

In [ ]:
flv = "tau"

if flv == "tau":
    identifier = "tau"
    summary_file = "full_nutau_sample.root"
elif flv == "mu":
    identifier = "numu"
    summary_file = "full_numu_sample.root"
elif flv == "nue":
    identifier = "showers"
    summary_file = "full_nue_sample.root"
    

In [ ]:
cluster = "woody"

if cluster == "woody":
    save_path = "/home/saturn/capn/mppi133h/master_thesis/tau_appearance/reconstruction_perfomance/plots"
    sys.path.append("/home/saturn/capn/mppi133h/master_thesis/tau_appearance/reconstruction_perfomance")
    path = "/home/wecapstor3/capn/mppi133h/ANTARES/mc"
elif cluster == "lyon":
    save_path = "/sps/km3net/users/mchadoli/master_thesis/tau_appearance/reconstuction_perfomance/plots/"
    sys.path.append("/sps/km3net/users/mchadoli/master_thesis/tau_appearance/reconstuction_perfomance")
    path = "/sps/km3net/users/mchadoli/ANTARES/"
    

import scripts.file_management as fm
import scripts.lib_masks as lm

In [ ]:
columns_energy = [
    "RunID",
    "EventID",
    "TriggCounter",
    "Frame",
    "interaction_type",
    "is_cc",
    "Type",
    "energy_true",
    "cos_zenith_true",
    "energy_aafit_dEdX_CEA",
    "energy_aafit_ANN_ECAP",
    "showerdusj_energy",
    "showertantra_energy",
]

In [ ]:
def rename_h5_df_cols(
    df,
    mapper={
        "TrigCount": "TriggCounter",
        "EventID": "Frame",
    },
):
    return df.rename(columns=mapper)

In [ ]:
# Load the data
print("Loading the data...")

# Load the dataframes
print("Importing into dataframes...")

nnfit_path = os.path.join(path, "nnfit_reco")

# Load nnfit reco files
nnfit_files = fm.list_files_with_pattern(nnfit_path, f"*{identifier}*")

In [ ]:
df_nnfit = fm.load_dataframes(nnfit_files, folder_path=path+"nnfit_reco")

In [ ]:
rootfile = os.path.join(path, f"extracted_merged/low_energy/{summary_file}")
dfnu = fm.rootfile_to_df(rootfile, columns=columns_energy)

In [ ]:
print("Renaming the columns...")
df_nnfit = rename_h5_df_cols(df_nnfit)

In [ ]:
# Merge the two dataframes
df = dfnu.merge(df_nnfit, on=["RunID","Frame","TriggCounter"], how="left")

print("Number of events in the merged dataframe: ", df.shape[0])
print("Number of NNfit events:", df_nnfit.shape[0])
print("Number of nutau events:", dfnu.shape[0])

In [ ]:
# Apply the masks
df = lm.apply_all_masks(df)

df_track_ma = lm.get_trackmask(df)
df_shower_cc_ma = lm.get_showermask_cc(df)
df_shower_nc_ma = lm.get_showermask_nc(df)

In [ ]:
# Add the energy columns
for column in ["NNFitTrack", "NNFitShower"]:
    df[f'{column}_Energy'] = df[f'{column}_Log10Energy'].apply(lambda x: 10**x)

In [ ]:
def plot_energy_reco(df, 
                     recos, 
                     mask,
                     plt_col,
                     plt_row, 
                     bins=100, 
                     xlim=(1e1, 1e2),
                     ylim=(10, 1e3),
                     xlabel="True Energy (GeV)", 
                     ylabel="Reconstructed Energy (GeV)",
                     path="/sps/km3net/users/mchadoli/tau_appearance/expected_events/plots/"
                     ):
    fig, ax = plt.subplots(nrows=plt_row, ncols=plt_col, figsize=(10, 10))
    
    #Flatten the axis
    ax = ax.flatten()
    
    for ax, reco in zip(ax, recos):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")        
            sns.histplot(data= df[mask][df[mask][reco]>0], x="energy_true", y=reco,
                         bins=bins, log_scale=(True,True), ax=ax, 
                         cbar=True,  cmap='viridis', cbar_kws=dict(shrink=.65),
                         )    
            ax.plot((0,1e2), (0,1e2), "k--", color="red",  alpha=0.75, zorder=2, linewidth=4) 
            ax.tick_params(axis='both', which='minor', rotation= 30, labelsize=10)
            ax.set(xlabel=None, ylabel=None)
            ax.set_title(f"{reco}", fontsize=22)
            if xlim:
                ax.set_xlim(xlim)
            if ylim:
                ax.set_ylim(ylim)
    fig.suptitle(f"True vs Reco for {df[mask]['Event type'].unique()[0]}", fontsize=24)
    fig.supxlabel(xlabel, fontsize=22)
    fig.supylabel(ylabel, fontsize=22)
    fig.tight_layout()
    fig.savefig(os.path.join(path, f"{df[mask]['Flavour type'].unique()[0]}_{df[mask]['Event type'].unique()[0]}_energy_reco_2dhisto.png"))

In [ ]:
# Plot for all masks 

def run_all_energy_reco_plots(df, recos, path):
    for mask in [df_track_ma, df_shower_cc_ma, df_shower_nc_ma]:
        plot_energy_reco(df, recos, mask, 2, 2, path=path)   

In [ ]:
run_all_energy_reco_plots(df, ["NNFitTrack_Energy", "NNFitShower_Energy", "energy_aafit_dEdX_CEA", "energy_aafit_ANN_ECAP"], save_path)

In [ ]:
# Calculate the relative error of the energy
track_energy_reco = ["energy_aafit_dEdX_CEA", "energy_aafit_ANN_ECAP", "NNFitTrack_Energy"]
shower_energy_reco = ["showerdusj_energy", "showertantra_energy", "NNFitShower_Energy"]

for algo in track_energy_reco + shower_energy_reco:        
    df[f"{algo}_relative_error"] = abs(df[f"{algo}"] - df["energy_true"])/df["energy_true"]

In [ ]:
# Create the true energy bins
energy_true_bins = np.linspace(10, 100, 25)
df["energy_true_bins"] = pd.cut(df["energy_true"], bins=energy_true_bins)

In [ ]:
def calculate_energy_reco_median(df, reco, energy_bins):
    return df.groupby("energy_true_bins")[reco].median().reset_index()

In [ ]:
# Create the median error dataframe
mean_error = pd.DataFrame()

columns_errors = ["median_error_nnfit_track", "median_error_nnfit_shower", "median_error_aafit_dEdX_CEA", "median_error_aafit_ANN_ECAP", "median_error_showerdusj_energy"]
df_cols = ["NNFitTrack_Energy_relative_error", "NNFitShower_Energy_relative_error", "energy_aafit_dEdX_CEA_relative_error", "energy_aafit_ANN_ECAP_relative_error", "showerdusj_energy_relative_error"]

dfs = df[(df["Flavour type"] == "tau") & (df["Event type"] == "showers_cc")]

# Group the data by the true energy bins
with warnings.catch_warnings():
    warnings.simplefilter("ignore")   
    for col, df_col in zip(columns_errors, df_cols):
        mean_error[col] = dfs.groupby("energy_true_bins").median()[df_col]
        

energy_true_bins_midpoints = dfs["energy_true_bins"].apply(lambda x: x.mid).unique()

midpoints = [(energy_true_bins[i] + energy_true_bins[i + 1]) / 2 for i in range(len(energy_true_bins) - 1)]
mean_error["midpoints"] = midpoints
mean_error.reset_index(inplace=True)
mean_error.head()


In [ ]:
dict_reco = {
    "median_error_nnfit_track": "NNFitTrack_Energy",
    "median_error_nnfit_shower": "NNFitShower_Energy",
    "median_error_aafit_dEdX_CEA": "energy_aafit_dEdX_CEA",
    "median_error_aafit_ANN_ECAP": "energy_aafit_ANN_ECAP",
    "median_error_showerdusj_energy": "showerdusj_energy"
}

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), sharex=True)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")        
    for col in columns_errors:
        sns.lineplot(data=mean_error, x=midpoints, y=mean_error[col]*100,   marker="o", markersize=10, linestyle="-", ax=ax, label=f"{dict_reco[col]}")
    ax.set_ylabel("Median relative absolute error " + r"$\frac{|E_{rec} - E_{true}|}{E_{true}}$" + " (%)", fontsize=20)
    ax.set_xlabel("True energy (GeV)", fontsize=20)
    ax.set_yscale("log")
    #ax.set_title("Median absolute relative error for energy reconstructions", fontsize=24)
    sns.move_legend(ax, "upper right", bbox_to_anchor=(1.01, 1.015), fontsize=20)   
    ax.tick_params(axis='both', which='major', labelsize=20)
    fig.tight_layout()
    fig.show()
fig.savefig(os.path.join(save_path, f"{identifier}_median_absolute_relative_error.png"))